Import Libraries & stuff

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as T
from torch.optim import Adam
from torchvision.datasets.mnist import MNIST
from torch.utils.data import DataLoader
import numpy as np

import torch
import torch.nn as nn
import torchvision.transforms as T
from torch.optim import Adam

from torchvision.datasets.mnist import MNIST
from torch.utils.data import DataLoader

from attention import VisionTransformer

Training Parameters

In [2]:
d_model = 9
n_classes = 10
img_size = (32,32)
patch_size = (16,16)
n_channels = 1
n_heads = 3
n_layers = 3
batch_size = 128
epochs = 5
alpha = 0.005

Loading MNIST Dataset

In [3]:
transform = T.Compose([
    T.Resize(img_size),
    T.ToTensor()
    ])

train_set = MNIST(
root="./datasets", train=True, download=True, transform=transform
)
test_set = MNIST(
root="./datasets", train=False, download=True, transform=transform
)

train_loader = DataLoader(train_set, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_set, shuffle=False, batch_size=batch_size)

Training

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device: ", device, f"({torch.cuda.get_device_name(device)})" if torch.cuda.is_available() else "")

transformer = VisionTransformer(d_model, n_classes, img_size, patch_size, n_channels, n_heads, n_layers).to(device)

optimizer = Adam(transformer.parameters(), lr=alpha)
criterion = nn.CrossEntropyLoss()

for epoch in range(epochs):

  training_loss = 0.0
  for i, data in enumerate(train_loader, 0):
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)

    optimizer.zero_grad()

    outputs = transformer(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    training_loss += loss.item()

  print(f'Epoch {epoch + 1}/{epochs} loss: {training_loss  / len(train_loader) :.3f}')

Using device:  cuda (NVIDIA GeForce RTX 4070)
Epoch 1/5 loss: 1.714
Epoch 2/5 loss: 1.569
Epoch 3/5 loss: 1.550
Epoch 4/5 loss: 1.541
Epoch 5/5 loss: 1.535


Testing

In [5]:
correct = 0
total = 0

with torch.no_grad():
  for data in test_loader:
    images, labels = data
    images, labels = images.to(device), labels.to(device)

    outputs = transformer(images)

    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
  print(f'\nModel Accuracy: {100 * correct // total} %')


Model Accuracy: 92 %
